<a href="https://colab.research.google.com/github/aswit3/Start_Your_NLP_Career/blob/master/elmo_with_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:

def data_prep_ELMo(train_x,train_y,test_x,test_y,max_len):

    INDEX_FROM = 3
    word_to_index = imdb.get_word_index()
    word_to_index = {k:(v+INDEX_FROM) for k,v in word_to_index.items()}

    word_to_index["<START>"] =1
    word_to_index["<UNK>"]=2

    index_to_word = {v:k for k,v in word_to_index.items()}

    sentences=[]
    for i in range(len(train_x)):
        temp = [index_to_word[ids] for ids in train_x[i]]
        sentences.append(temp)

    test_sentences=[]
    for i in range(len(test_x)):
        temp = [index_to_word[ids] for ids in test_x[i]]
        test_sentences.append(temp)

    train_text = [' '.join(sentences[i][:max_len]) for i in range(len(sentences))]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    train_label = train_y.tolist()

    test_text = [' '.join(test_sentences[i][:500]) for i in range(len(test_sentences))]
    test_text = np.array(test_text , dtype=object)[:, np.newaxis]
    test_label = test_y.tolist()

    return train_text,train_label,test_text,test_label


In [0]:
def load_data(vocab_size,max_len):
    """
        Loads the keras imdb dataset

        Args:
            vocab_size = {int} the size of the vocabulary
            max_len = {int} the maximum length of input considered for padding

        Returns:
            X_train = tokenized train data
            X_test = tokenized test data

    """
    INDEX_FROM = 3

    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

    (X_train,y_train),(X_test,y_test) = imdb.load_data(num_words = vocab_size,index_from = INDEX_FROM)

    # restore np.load for future normal usage
    np.load = np_load_old

    print(X_train.shape, len(X_test), y_train.shape, len(y_test), "#####################################")
         
    return X_train,X_test,y_train,y_test

In [0]:
def data_prep_ELMo(train_x,train_y,test_x,test_y,max_len):

    INDEX_FROM = 3
    word_to_index = imdb.get_word_index()
    word_to_index = {k:(v+INDEX_FROM) for k,v in word_to_index.items()}

    word_to_index["<START>"] =1
    word_to_index["<UNK>"]=2

    index_to_word = {v:k for k,v in word_to_index.items()}

    sentences=[]
    for i in range(len(train_x)):
        temp = [index_to_word[ids] for ids in train_x[i]]
        sentences.append(temp)

    test_sentences=[]
    for i in range(len(test_x)):
        temp = [index_to_word[ids] for ids in test_x[i]]
        test_sentences.append(temp)

    train_text = [' '.join(sentences[i][:max_len]) for i in range(len(sentences))]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    train_label = train_y.tolist()

    test_text = [' '.join(test_sentences[i][:500]) for i in range(len(test_sentences))]
    test_text = np.array(test_text , dtype=object)[:, np.newaxis]
    test_label = test_y.tolist()

    return train_text,train_label,test_text,test_label

In [5]:
embed_dim = 300
split_ratio= 0.33
max_len= 200
vocab_size= 100
trainable_param= False
workers = 3,
window = 1


x_train,x_test,y_train,y_test = load_data(vocab_size,max_len)

train_text,train_label,test_text,test_label = data_prep_ELMo(x_train,y_train,x_test,y_test,max_len)



(25000,) 25000 (25000,) 25000 #####################################


In [0]:
# "epochs":1,
# "batch_size":1024,
# "loss":"binary_crossentropy",
# "optimizer":"adam",
# "metrics":["accuracy"]
      
def ELMoEmbedding(x):
    elmo_model = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
    print(elmo_model, "elmo loaded successfully")
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

  
def Classification_model_with_ELMo(X_train_pad,y_train,X_test_pad,y_test):
    input_text = layers.Input(shape=(1,), dtype=tf.string)
    embed_seq = layers.Lambda(ELMoEmbedding, output_shape=(1024,))(input_text)
    
    print(embed_seq, input_text)
    
    x = Dense(256,activation ="relu")(embed_seq)
    preds = Dense(1,activation="sigmoid")(x)
    model = Model(input_text,preds)


    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

    model.fit(X_train_pad[:100],y_train[:100],epochs=1,batch_size=64,validation_data=(X_test_pad[:10],y_test[:10]))

    predictions = model.predict(X_test_pad)
    predictions = [0 if i<0.5 else 1 for i in predictions]
    print("Accuracy: ",accuracy_score(y_test,predictions))
    print("Classification Report: ",classification_report(y_test,predictions))

    return model


In [0]:
model = Classification_model_with_ELMo(train_text,train_label,test_text,test_label)
print(model.summary())

W0717 15:03:50.759687 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 15:03:50.776861 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



<tensorflow_hub.module.Module object at 0x7f40222b4748> elmo loaded successfully


W0717 15:03:52.186433 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 15:03:52.217591 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0717 15:03:52.242604 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0717 15:03:52.250121 139914399782784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, whi

Tensor("lambda_1/module_apply_default/pooling/truediv:0", shape=(?, 1024), dtype=float32) Tensor("input_1:0", shape=(?, 1), dtype=string)


W0717 15:03:52.434345 139914399782784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 100 samples, validate on 10 samples
Epoch 1/1
100/100 [==============================] - 164s 2s/step - loss: 0.7625 - acc: 0.5000 - val_loss: 0.7457 - val_acc: 0.4000
